In [1]:
from testdataextractor.testdataextractor.extractor import Extractor
from summpy.summpy import lexrank
import pandas as pd

## Get some data

In [2]:

testarticles = [1957284403,1965754064,233465322,283147769,362778020,37793736,389321649,540607195,60134403,887344770, ]
all_articles = []
all_sets_sentences = []
for art in testarticles:
    ext = Extractor("../test_data/{0}.ofs.gold.xml".format(art))
    article = ext.extract()
    all_articles.append(article)
    df_article = pd.DataFrame.from_dict(article['sentences'], orient='index')
    sorted_indexes = [ "s{0}".format(x) for x in range(190)]
    sentences = list(df_article.ix[sorted_indexes, 'text'])
    print len(sentences), "sentences extracted."
    if df_article.ix['s2', 'text'] == sentences[2]:
        print "Extracted list of sentences is in a proper order."
        all_sets_sentences.append(sentences)
    else:
        print "Extracted list of sentences is unordered."

190 sentences extracted.
Extracted list of sentences is in a proper order.
190 sentences extracted.
Extracted list of sentences is in a proper order.
190 sentences extracted.
Extracted list of sentences is in a proper order.
190 sentences extracted.
Extracted list of sentences is in a proper order.
190 sentences extracted.
Extracted list of sentences is in a proper order.
190 sentences extracted.
Extracted list of sentences is in a proper order.
190 sentences extracted.
Extracted list of sentences is in a proper order.
190 sentences extracted.
Extracted list of sentences is in a proper order.
190 sentences extracted.
Extracted list of sentences is in a proper order.
190 sentences extracted.
Extracted list of sentences is in a proper order.


## Feed data into lexrank
The summpy MIT licensed repo used for this task, returns a tuple with a dictionary with sentences index + score, and the similarity_matrix.

In [3]:
all_ranked_sentences = []
all_matrxs = []
for sentences in all_sets_sentences:
    ranked_sentences, similarity_mtrx = lexrank.lexrank(sentences)
    all_ranked_sentences.append(ranked_sentences)
    all_matrxs.append(similarity_mtrx)

AttributeError: 'float' object has no attribute 'strip'

## See the data

In [ ]:
df_ranked_sentences = pd.DataFrame.from_dict(ranked_sentences,
                                             orient='index')
df_similarity_mtrx = pd.DataFrame(similarity_mtrx)



In [ ]:
mean_score = df_ranked_sentences.mean(axis=0)
top_sent = list(df_ranked_sentences[df_ranked_sentences > mean_score].dropna().index)
bottom_sent = list(df_ranked_sentences[df_ranked_sentences < mean_score].dropna().index)

print "{0} top sentences and {1} bottom_sentences".format(len(top_sent), len(bottom_sent))

## Compare with link information
* How many of the links are made entirely of top sentences?
* How many are made of top and bottom sentences? 
* How many of just bottom sentences?

Answering these questions will probably allow me to know if the saliency of these sentences given by lexrank is a good feature for finding links, or if it is just useful for finding the most important ones and that's it.

In [ ]:
top_sent_set = { "s{0}".format(s) for s in top_sent }
bot_sent_set = { "s{0}".format(s) for s in bottom_sent }

both_top = 0
one_top = 0
both_bottom = 0
other = 0
link_dicts = article['links'].values()
total = len(link_dicts)*1.0
for l in link_dicts:
    s_art = l['art_sentence']
    s_com = l['com_sentence']
    if s_art in top_sent_set and s_com in top_sent_set:
        both_top += 1
        one_top += 1
    elif s_art in bot_sent_set and s_com in bot_sent_set:
        both_bottom += 1
    else:
        other += 1
        one_top += 1

print "TOP: {0}, BOTTOM: {1}, MIXED: {2}, AT LEAST ONE TOP: {3}"\
.format(both_top/total,both_bottom/total,other/total, one_top/total)

Art1 = .8571
Art2 = .7549
Art3 = 